# Setup Git-crypt with empty directory

In [198]:
DEBUG=True

# feature flags to debug
SETUP=False
UNLOCK=True
INCR_EXPERIMENTAL=False
#ref: https://stephanosterburg.gitbook.io/scrapbook/data-science/jupiter-notebook-tips-and-tricks

In [191]:
%alias gcrypt docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s
%store gcrypt
if DEBUG:
    %alias

Alias stored: gcrypt (docker run -t -v %s:/repo xueshanf/git-crypt git-crypt %s)
Total number of aliases: 16


In [192]:
if SETUP:
    pass
    %gcrypt /Users/kenneyhe/Desktop/data-science status -e
    %gcrypt /Users/kenneyhe/Desktop/data-science init
    %gcrypt /Users/kenneyhe/Desktop/data-science export-key git-crypt-ds-key
    !mv git-crypt-ds-key ..
    !git add .gitattributes
    # if using osx to add
    !brew install --build-from-source git-crypt
    %gcrypt /Users/kenneyhe/Desktop/data-science unlock ../git-crypt-ds-key

if UNLOCK:
    !docker run -t -v $PWD:/repo  -v /Users/kenneyhe/Desktop/git-crypt-ds-key:/repo/key xueshanf/git-crypt git-crypt unlock key

In [193]:
!git commit -m "Tell git-crypt to encrypt secrets.yaml"

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Init.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/Api-checkpoint.ipynb
	.ipynb_checkpoints/BackupHelper-checkpoint.ipynb
	key

no changes added to commit (use "git add" and/or "git commit -a")


In [194]:
# %load secrets.yaml
PASS="951766d1-4c6d-44be-b610-ee0eba6f372d"
USER="kenney"


In [195]:
# %load api.py
import papermill as pm
import datetime, uuid, os, sys

timestamp=datetime.date.today().strftime("%Y%m%d%H%sb")
uuid_str=str(uuid.uuid1())[0:7]

import ipywidgets as widgets

# default feature flags
INCR_EXPERIMENTAL=False
DEBUG=False
RESTORE=False
DEP_NEED=False

def sync_mobile(dir):
    '''
        DESCRIPTION: sync latest to the current dated tag
        PARAM: dir_list is list of directorys under Desktop
               gs://.../.. can be used instead if upload output to
               s3
    '''    
    if INCR_EXPERIMENTAL:
        dir = dir.lower()
        # match backup helper convention
        TAG_NAME="%s-%s" % (dir, timestamp)

        status = os.system('''
                    docker login -u {user} --password {dpass};
                    docker tag kenney/{tname} kenney/mobile:{tname};
                    docker tag kenney/{tname} kenney/mobile:{tdir}-latest;            
                    docker push kenney/mobile:{tname} && docker push kenney/mobile:{tdir}-latest;         
                    '''.format(tname=TAG_NAME, tdir=dir, user=USER, dpass=PASS))
        if DEBUG:
            print('%s has status %d: done' % (TAG_NAME, status))
        assert status == 0, "failed"


def backup(dir_list, isFull=False, generate=False):
    '''
        DESCRIPTION: backups the dir_lists and upload to docker
        PARAM: dir_list is list of directorys under Desktop
               gs://.../.. can be used instead if upload output to
               s3
        EXCEPTIONS: throw exception up to caller if unexpected
    '''
    for dir in dir_list:
        uuid_str=str(uuid.uuid1())[0:7]
        output="output_%s_run_backup_%s.ipynb" % (timestamp, uuid_str)
        try:
            pm.execute_notebook("BackupHelper.ipynb",
                                output,
                                {"PASS": PASS,
                                 "DIR": dir,
                                 "DEBUG": False,
                                 "FULL": isFull,
                                 "GEN": generate,
                                 "progress_bar": True,
                                 "log_output": True,
                                 "report_mode": True,
                                }
                               )
        except pm.PapermillExecutionError:
            print("failed to run %s" %(dir))
            if DEBUG:
                print(uuid_str, '=>', output)
            sys.exit(1)
        print('%s is done' % (dir))

        sync_mobile(dir)


def restore(dir_dict):
    '''
        DESCRIPTION: restores the dir_lists and upload to docker
               assuming all directories are made to be public
        PARAM: dir_dict is dictionary of directory name with tags
               under Desktop
               gs://.../.. can be used instead if upload output to
               s3
    '''
    for dir, tag in dir_dict.items():
        uuid_str=str(uuid.uuid1())[0:7]
        output="output_%s_run_restore_%s.ipynb" % (timestamp, uuid_str)
        try:
            pm.execute_notebook("RestoreBackup.ipynb",
                                output,
                                {"PASS": PASS, 
                                 "DIR": dir,
                                 "TAG": tag,
                                 "DEBUG": True}
                               )
        except pm.PapermillExecutionError:
            print("failed to run %s" %(dir))
            if DEBUG:
                print(uuid_str, '=>', output)
            sys.exit(1)
        except:
            print("unknown error")
            sys.exit(1)


def cleanup():
    '''
        DESCRIPTION: cleanup pyc, .DS_Store, __pycache__ files and
               prune all images unused
        PARAM: None   
    '''
    status = os.system('''
                find . -iname \*pyc -exec rm {} \;
                find . -iname .DS_Store -exec rm {} \;
                find . -iname __pycache__ -exec rmdir {} \;
                docker system prune -af;
                ''')
    assert status == 0, "failed"


def backup_incre(dir_list):
    '''
        DESCRIPTION: use docker-compose to do incremental and faster
                backup.  Must have INCR_EXPERIMENTAL as True to 
                speedup. Uses latest tag to speed up
        PARAM: None
        RETURN: if success returns 0, otherwise non-zero
    '''
    try:
        backup(dir_list, isFull=False, generate=True)
    except:
        return -1
    return 0


def widget_run_default_backup(pos):
    '''
        DESCRIPTION: use button to trigger backup from the blist selected
        PARAM: None      
    '''

    selected = blist.trait_values()['value']
    if DEBUG:
        print(selected)
    pos.set_trait('button_style', 'info')
    pos.set_trait('description', 'In progress...') 
    
    if backup_incre(selected) == 0:
        pos.set_trait('button_style', 'success')
        pos.set_trait('description', 'Done')
    else:
        pos.set_trait('button_style', 'danger')
        pos.set_trait('description', 'Failed')


In [199]:
if DEBUG:
    import sys
    backup(["Tests"])

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.




failed to run Tests
bbd507d => output_20210104001609747200b_run_backup_bbd507d.ipynb
Traceback (most recent call last):
  File "<ipython-input-195-2b163a622ece>", line 51, in backup
    pm.execute_notebook("BackupHelper.ipynb",
  File "/usr/local/lib/python3.8/site-packages/papermill/execute.py", line 118, in execute_notebook
    raise_for_execution_errors(nb, output_path)
  File "/usr/local/lib/python3.8/site-packages/papermill/execute.py", line 230, in raise_for_execution_errors
    raise error
papermill.exceptions.PapermillExecutionError: 
---------------------------------------------------------------------------
Exception encountered at "In [5]":
---------------------------------------------------------------------------
HTTPError                                 Traceback (most recent call last)
/usr/local/lib/python3.8/site-packages/docker/api/client.py in _raise_for_status(self, response)
    267         try:
--> 268             response.raise_for_status()
    269         excep

TypeError: object of type 'NoneType' has no len()

In [197]:
%gcrypt /Users/kenneyhe/Desktop/data-science lock --force
!cat secrets.yaml

 GITCRYPT }��S~Ƣ��4�6t�p�t�,���-#Y*k�!�o�2�FHA*c��!K,$���^��&��n`����L

In [146]:
%gcrypt /Users/kenneyhe/Desktop/data-science status

not encrypted: .ipynb_checkpoints/Api-checkpoint.ipynb
not encrypted: .ipynb_checkpoints/BackupHelper-checkpoint.ipynb
not encrypted: .gitattributes
not encrypted: .gitignore
not encrypted: .ipynb_checkpoints/Init-checkpoint.ipynb
not encrypted: BackupHelper.ipynb
not encrypted: Init.ipynb
not encrypted: LICENSE
not encrypted: README.md
not encrypted: Tests/Dockerfile
not encrypted: Tests/Widget%20List.ipynb
not encrypted: api.py
    encrypted: secrets.yaml
